In [1]:
import numpy as np
import sys
import os
from typing import Sequence
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from gradient_descendent import gradient_descendent
from normalize import MinMaxNormalizer, StandardScaler
from loss_fn_tarefa2 import make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function
from plots.plots_tarefa2 import plot_best_by_loss

# Ajuste de curva por otimização

## Carregar os dados

In [2]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho2dados.xlsx')

df.head()

,x,y,z
0,-5.0,-5.0,-458.963629
1,-5.0,-3.8,-520.361381
2,-5.0,-2.6,-593.039231
3,-5.0,-1.4,-606.776605
4,-5.0,-0.2,-657.401892


## EDA dados

In [3]:
df.describe()

,x,y,z
count,81.000000,81.000000,81.000000
mean,-0.200000,-0.200000,36.990573
std,3.117691,3.117691,294.680404
min,-5.000000,-5.000000,-657.401892
25%,-2.600000,-2.600000,-42.780939
50%,-0.200000,-0.200000,59.403645
75%,2.200000,2.200000,170.599518
max,4.600000,4.600000,614.700100


In [4]:
df.isna().sum()

x    0
y    0
z    0
dtype: int64

In [5]:
df_pivot = df.pivot(index='y', columns='x', values='z')

# Gerar a malha
x_axis = df_pivot.columns.values
y_axis = df_pivot.index.values
z_grid = df_pivot.values

fig = go.Figure()

# Adicionar a Superfície
fig.add_trace(go.Surface(
    x=x_axis,
    y=y_axis,
    z=z_grid,
    colorscale='Viridis',
    opacity=0.9,
    contours_z=dict(show=True, usecolormap=True, project_z=True, highlightcolor="white"),
    name='Superfície dos Dados'
))

# Adicionar os Pontos de Dados Originais
fig.add_trace(go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(size=3, color='red', symbol='circle'),
    name='Pontos de Dados Originais'
))

# Melhorar o Layout
fig.update_layout(
    title=dict(text='z = f(x, y)', x=0.5),
    scene=dict(
        xaxis_title='Eixo X',
        yaxis_title='Eixo Y',
        zaxis_title='Eixo Z (Valor)'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

fig.show()

In [6]:
X = np.column_stack([df['x'] ** 3, df['y'] ** 2, np.ones(len(df['x']))])

true_weights = np.linalg.inv((X.T @ X) + (X.T @ X).T) @ (2 * X.T @ df['z'])
true_weights

array([ 5.0081046 ,  5.73419945, 10.59963756])

## Calcular as funções de perda

### Configurações

In [7]:
def make_function(w):
    def f(x, y):
        if np.isscalar(x) and np.isscalar(y):
            return w @ np.array([x**3, y**2, 1])
        else:
            return np.array([w @ np.array([xi**3, yi**2, 1]) for xi, yi in zip(x, y)])
    return f

features = df[['x', 'y']]
y = df['z']

# Criar os objetos para Normalização e Padronização
min_max_scaler = MinMaxNormalizer(-1, 1)
standard_scaler = StandardScaler()

# Cria as cópias dos dados para normalização e padronização
cubed_squared_features = features.copy()  # Mantém os dados originais para comparação
cubed_squared_features.loc[:, 'x'] = cubed_squared_features['x'] ** 3
cubed_squared_features.loc[:, 'y'] = cubed_squared_features['y'] ** 2
features_normalized = cubed_squared_features.copy()
features_standardized = cubed_squared_features.copy()

# Ajusta os normalizadores e padronizadores aos dados
min_max_scaler.fit(features_normalized)
standard_scaler.fit(features_standardized)

# Normaliza e Padroniza os dados
features_normalized = min_max_scaler.normalize(features_normalized)
features_standardized = standard_scaler.normalize(features_standardized)

# Listas para iteração
features_list = [features, features_normalized, features_standardized]
features_names = ['Original', 'Normalized', 'Standardized']
loss_fn_names = ['MSE', 'MAE', 'RMSE']
loss_fn_makers = [make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function]

learning_rates = [0.001, 0.0001, 0.00001]
initial_weights = [
    np.zeros(3),  # Zero sempre seguro
    np.array([0.1, 0.1, 0.1]),  # Valores pequenos
    np.random.randn(3) * 0.01  # Inicialização normal pequena
]
n_iterations = 10000
tolerance = 1e-6

### Rodar os experimentos

In [8]:
dict_results = {}

for feature_set, feature_name in zip(features_list, features_names):
    x_data = feature_set['x'].values
    y_data = feature_set['y'].values
    z_data = y.values

    x_data_orig = features['x'].values
    y_data_orig = features['y'].values

    for loss_fn_name, loss_fn_maker in zip(loss_fn_names, loss_fn_makers):
        loss_function, grad_loss_function = None, None

        if feature_name == 'Original':
            loss_function, grad_loss_function = loss_fn_maker(x_data ** 3, y_data ** 2, z_data)
        else:
            loss_function, grad_loss_function = loss_fn_maker(x_data, y_data, z_data)

        for lr in learning_rates:
            for initial_w in initial_weights:
                try:
                    weights, losses, n_iters = gradient_descendent(
                        initial_w, loss_function, grad_loss_function,
                        learning_rate=lr, max_iter=n_iterations, tolerance=tolerance, stopping_criteria=[1, 2, 3]
                    )

                    # Verificar se há overflow
                    if np.any(np.isnan(weights[-1])) or np.any(np.isinf(weights[-1])):
                        print(f"Overflow detectado para LR={lr}, Initial_W={initial_w}")
                        continue

                except Exception as e:
                    print(f"Erro para LR={lr}, Initial_W={initial_w}: {e}")
                    continue

                if feature_name == 'Standardized':
                    weights[-1] = standard_scaler.desnormalize_weights(weights[-1])
                elif feature_name == 'Normalized':
                    weights[-1] = min_max_scaler.desnormalize_weights(weights[-1])

                function_aprox = make_function(weights[-1])

                mse_final = np.mean((z_data - function_aprox(x_data_orig, y_data_orig)) ** 2)
                rmse_final = np.sqrt(mse_final)
                mae_final = np.mean(np.abs(z_data - function_aprox(x_data_orig, y_data_orig)))

                key = (feature_name, loss_fn_name, lr, tuple(initial_w))
                dict_results[key] = {
                    'weights': weights,
                    'losses': losses,
                    'n_iters': n_iters,
                    'mse_final': mse_final,
                    'rmse_final': rmse_final,
                    'mae_final': mae_final
                }

df_result = pd.DataFrame([
    {
        'Feature_Set': key[0],
        'Loss_Function': key[1],
        'Learning_Rate': key[2],
        'Initial_Weights': key[3],
        'Final_Weights': value['weights'][-1],
        'Final_Loss': value['losses'][-1],
        'MSE_Final': value['mse_final'],
        'RMSE_Final': value['rmse_final'],
        'MAE_Final': value['mae_final'],
        'Iterations': value['n_iters']
    }
    for key, value in dict_results.items()
])

df_result_to_save = df_result.copy()
df_result_to_save['Learning_Rate'] = df_result_to_save['Learning_Rate'].apply(lambda x: f"{float(x):.6g}")
df_result_to_save['Initial_Weights'] = df_result_to_save['Initial_Weights'].apply(lambda x: np.array(x).tolist())
df_result_to_save['Final_Weights'] = df_result_to_save['Final_Weights'].apply(lambda x: np.array(x).tolist())

df_result_to_save['Initial_Weights'] = df_result_to_save['Initial_Weights'].apply(lambda x: str(x))
df_result_to_save['Final_Weights'] = df_result_to_save['Final_Weights'].apply(lambda x: str(x))

df_result_to_save.to_excel('../output/tarefa2_results.xlsx', index=False, float_format="%.6g")

df_result.sort_values(by='MSE_Final').groupby('Loss_Function').head(3).reset_index(drop=True).drop(columns=['Final_Loss'])

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\.venv\Lib\site-packages\numpy\_core\_methods.py:134: RuntimeWarning:

overflow encountered in reduce

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:14: RuntimeWarning:

overflow encountered in square

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\libs\gradient_descendent.py:63: RuntimeWarning:

invalid value encountered in scalar subtract

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:19: RuntimeWarning:

overflow encountered in matmul

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:19: RuntimeWarning:

invalid value encountered in matmul

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\libs\gradient_descendent.py:49: RuntimeWarning:

invalid value encountered in subtract



Overflow detectado para LR=0.001, Initial_W=[0. 0. 0.]
Overflow detectado para LR=0.001, Initial_W=[0.1 0.1 0.1]
Overflow detectado para LR=0.001, Initial_W=[-0.0059059  -0.00608641  0.00537935]


,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Standardized,MSE,0.001,"(0.0, 0.0, 0.0)","[5.007836664166791, 5.7338926715385785, 10.599...",372.092180,19.289691,15.294893,4913
1,Standardized,MSE,0.001,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.0078366586745755, 5.733892633296255, 10.599...",372.092180,19.289691,15.294893,4913
2,Standardized,MSE,0.001,"(0.1, 0.1, 0.1)","[5.007836220412937, 5.733892686337784, 10.5990...",372.092181,19.289691,15.294892,4912
3,Original,RMSE,0.001,"(0.1, 0.1, 0.1)","[5.001863826357645, 6.213147485618186, 2.36617...",401.670789,20.041726,15.672183,10000
4,Original,RMSE,0.001,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.001807429749923, 6.2174756455022235, 2.2917...",402.207801,20.055119,15.682531,10000
5,Original,RMSE,0.001,"(0.0, 0.0, 0.0)","[5.001803905449378, 6.217746118086512, 2.28712...",402.241520,20.055960,15.683177,10000
6,Original,MAE,0.001,"(0.0, 0.0, 0.0)","[5.01816187654278, 6.283904691357363, 1.135543...",412.167477,20.301908,15.825815,4949
7,Original,MAE,0.001,"(0.1, 0.1, 0.1)","[5.0166004938267905, 6.285862716048745, 1.1116...",412.194659,20.302578,15.829857,4256
8,Original,MAE,0.001,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.016100918186136, 6.299920993932606, 0.74830...",415.254500,20.377794,15.880197,2127


## Resultados

### MSE

In [9]:
_ = plot_best_by_loss(df_result, "MSE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado MSE:
Feature Set: Standardized
Learning Rate: 0.001
Final Loss: 372.09217995127085
Pesos: [ 5.00783666  5.73389267 10.59907048]


In [10]:
df_result_mse = df_result[df_result['Loss_Function'] == 'MSE'].sort_values(by='MSE_Final').drop(columns='Final_Loss').reset_index(drop=True)
df_result_mse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Standardized,MSE,0.00100,"(0.0, 0.0, 0.0)","[5.007836664166791, 5.7338926715385785, 10.599...",372.092180,19.289691,15.294893,4913
1,Standardized,MSE,0.00100,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.0078366586745755, 5.733892633296255, 10.599...",372.092180,19.289691,15.294893,4913
2,Standardized,MSE,0.00100,"(0.1, 0.1, 0.1)","[5.007836220412937, 5.733892686337784, 10.5990...",372.092181,19.289691,15.294892,4912
3,Original,MSE,0.00010,"(0.1, 0.1, 0.1)","[5.004883393524895, 5.981410684267237, 6.34990...",379.972200,19.492876,15.316278,10000
4,Original,MSE,0.00010,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.004855368452038, 5.983561469589042, 6.31292...",380.109916,19.496408,15.318365,10000
5,Original,MSE,0.00010,"(0.0, 0.0, 0.0)","[5.004853554896708, 5.983700650964379, 6.31053...",380.118869,19.496637,15.318501,10000
6,Original,MSE,0.00001,"(0.1, 0.1, 0.1)","[5.001058032080917, 6.274988199168381, 1.30309...",409.802224,20.243572,15.820022,10000
7,Original,MSE,0.00001,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.000996725627445, 6.279693165358287, 1.22221...",410.461252,20.259843,15.831270,10000
8,Original,MSE,0.00001,"(0.0, 0.0, 0.0)","[5.0009927583711935, 6.279997632595718, 1.2169...",410.504096,20.260901,15.831998,10000
9,Standardized,MSE,0.00010,"(0.1, 0.1, 0.1)","[4.330702099721204, 4.959904277874321, 9.16498...",1959.165260,44.262459,33.734391,10000


### RMSE

In [11]:
_ = plot_best_by_loss(df_result, "RMSE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado RMSE:
Feature Set: Original
Learning Rate: 0.001
Final Loss: 20.041726203384506
Pesos: [5.00186383 6.21314749 2.36617984]


In [12]:
df_result_rmse = df_result[df_result['Loss_Function'] == 'RMSE'].sort_values(by='RMSE_Final').drop(columns='Final_Loss').reset_index(drop=True)
df_result_rmse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,RMSE,0.00100,"(0.1, 0.1, 0.1)","[5.001863826357645, 6.213147485618186, 2.36617...",401.670789,20.041726,15.672183,10000
1,Original,RMSE,0.00100,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.001807429749923, 6.2174756455022235, 2.2917...",402.207801,20.055119,15.682531,10000
2,Original,RMSE,0.00100,"(0.0, 0.0, 0.0)","[5.001803905449378, 6.217746118086512, 2.28712...",402.241520,20.055960,15.683177,10000
3,Original,RMSE,0.00010,"(0.1, 0.1, 0.1)","[5.000111977584725, 6.291770872525102, 0.60501...",415.759530,20.390182,15.922895,10000
4,Original,RMSE,0.00010,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.0000071815994165, 6.2948021710347986, 0.516...",416.554488,20.409667,15.936537,10000
5,Original,RMSE,0.00010,"(0.0, 0.0, 0.0)","[5.000004796867698, 6.295228044651212, 0.51059...",416.602406,20.410840,15.937325,10000
6,Original,RMSE,0.00001,"(0.1, 0.1, 0.1)","[4.4955877486300215, 0.7459156620336218, 0.137...",6136.013689,78.332711,63.852982,10000
7,Original,RMSE,0.00001,"(0.0, 0.0, 0.0)","[4.4513994783966515, 0.6352689964662742, 0.036...",6475.589258,80.471046,65.569009,10000
8,Original,RMSE,0.00001,"(-0.0059058966286713, -0.006086413474760214, 0...","[4.448775131355467, 0.6285362785377299, 0.0421...",6495.287230,80.593345,65.663571,10000
9,Standardized,RMSE,0.00100,"(0.1, 0.1, 0.1)","[0.17140066795488465, 0.20567465464406204, 0.3...",81282.131892,285.100214,209.298387,10000


### MAE

In [13]:
_ = plot_best_by_loss(df_result, "MAE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado MAE:
Feature Set: Original
Learning Rate: 0.001
Final Loss: 15.825814843820108
Pesos: [5.01816188 6.28390469 1.13554321]


In [14]:
df_result_mae = df_result[df_result['Loss_Function'] == 'MAE'].sort_values(by='MAE_Final').drop(columns='Final_Loss').reset_index(drop=True)
df_result_mae

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,MAE,0.00100,"(0.0, 0.0, 0.0)","[5.01816187654278, 6.283904691357363, 1.135543...",412.167477,20.301908,15.825815,4949
1,Original,MAE,0.00100,"(0.1, 0.1, 0.1)","[5.0166004938267905, 6.285862716048745, 1.1116...",412.194659,20.302578,15.829857,4256
2,Original,MAE,0.00100,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.016100918186136, 6.299920993932606, 0.74830...",415.254500,20.377794,15.880197,2127
3,Original,MAE,0.00010,"(0.1, 0.1, 0.1)","[5.007548316049207, 6.24645397530912, 0.644371...",416.256517,20.402365,15.919502,9749
4,Original,MAE,0.00010,"(-0.0059058966286713, -0.006086413474760214, 0...","[5.007101698432865, 6.247812500105955, 0.55978...",417.055668,20.421941,15.933242,9809
5,Original,MAE,0.00010,"(0.0, 0.0, 0.0)","[5.00462019753064, 6.232967753086868, 0.549053...",417.440771,20.431367,15.945192,9521
6,Original,MAE,0.00001,"(0.1, 0.1, 0.1)","[3.617496393086131, 0.6403988641975492, 0.1449...",11309.454215,106.345918,85.443627,10000
7,Original,MAE,0.00001,"(0.0, 0.0, 0.0)","[3.5275408809873916, 0.5369946172840103, 0.044...",12301.337809,110.911396,89.000437,10000
8,Original,MAE,0.00001,"(-0.0059058966286713, -0.006086413474760214, 0...","[3.5222546209019323, 0.530696480352461, 0.0499...",12360.577648,111.178135,89.204714,10000
9,Standardized,MAE,0.00100,"(0.1, 0.1, 0.1)","[0.11964077921483865, 0.16042554786508506, 2.8...",82820.985065,287.786353,210.517090,10000
